# Generate intraday bars using bars data service - Python

### Overview
Bars are widely used to illustrate price movements for an instrument over a look back period. Commonly used in financial analysis and trading strategies as per a deicion making technical indicator.

This sample demonstrates how to directly request and plot `intraday bars` with a **customizable** time range sampling.<br>

It enables to retrieve **on-demand** data by calling a dedicated bars service on a tick by tick data store. 

### Inputs/outputs
Bars sample requires instrument's identifier, date time intervals as per inputs. It returns open-high-low-close (OHLC) and volume.
This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

### Services used
This sample uses *gRPC requests* in order to retrieve intraday bars from the dedicated hosted service. The queried endpoint in this script are:
* *TickBarsService*: to directly retrieve tick bars objects from the server

### Modules required
1. Systemathics:
    * *systemathics.apis.services.tick_analytics.v1*
    * *systemathics.apis.type.shared.v1*
    * *google.type*
2. Open source:
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*
    * *plotly* and *mplfinance* as per display packages

***

# Run intraday bars sample

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas plotly mplfinance systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
from datetime import timedelta
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import systemathics.apis.type.shared.v1.identifier_pb2 as identifier
import systemathics.apis.type.shared.v1.constraints_pb2 as constraints
import systemathics.apis.type.shared.v1.date_interval_pb2 as dateinterval
import systemathics.apis.type.shared.v1.time_interval_pb2 as timeinterval
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2 as tick_bars
import systemathics.apis.services.tick_analytics.v1.tick_bars_pb2_grpc as tick_bars_service
import systemathics.apis.helpers.token_helpers as token_helpers
import systemathics.apis.helpers.channel_helpers as channel_helpers

### Step 2: Retrieve authentication token
The following code snippet sends authentication request and print token to console output in order to process the upcomming *gRPC queries*.

In [ ]:
token = token_helpers.get_token()
display(token)

### Step 3: Create and process request
To request *tick bars service*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Bars request parameters

#### 3.1 Instrument selection

In [ ]:
# set instrument identifier: exchange + ticker
ticker = 'AAPL'
exchange = 'BATS'

#### 3.2 Bars calculation parameters

In [ ]:
# set the bar duration
sampling = 5 * 60

# set the bar calculation field
field = tick_bars.BAR_PRICE_TRADE 

#### 3.3 Time period delimitation

In [ ]:
# create time intervals (we are using Google date format)
today = datetime.today()
start = today - timedelta(days=50)

date_interval = dateinterval.DateInterval(
    start_date = date.Date(year = start.year, month = start.month, day = start.day), 
    end_date = date.Date(year = today.year, month = today.month, day = today.day)
)

# build the market data request time interval (we are using Google time format)
# UTC time zone
time_interval = timeinterval.TimeInterval(
    start_time = timeofday.TimeOfDay(hours = 15, minutes = 0, seconds = 0), 
    end_time = timeofday.TimeOfDay(hours = 16, minutes = 0, seconds = 0)
)

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and ensure that the reply is not empty:

In [ ]:
# generate constraints based on the previous time selection
constraints = constraints.Constraints(
    date_intervals = [date_interval],
    time_intervals = [time_interval],
)

In [ ]:
# generate tick bars request
request = tick_bars.TickBarsRequest(
    identifier = identifier.Identifier(exchange = exchange, ticker = ticker),
    constraints = constraints,
    sampling = duration.Duration(seconds = sampling),
    field = field
)

In [ ]:
try:
    # open a gRPC channel
    with channel_helpers.get_grpc_channel() as channel:  
        
        # instantiate the tick bars service
        service = tick_bars_service.TickBarsServiceStub(channel)
        
        # process the tick bars request
        bars = []
        metadata = [('authorization', token)]
        for bar in service.TickBars(request=request, metadata=metadata):
                bars.append(bar)
except grpc.RpcError as e:
    display(e.code().name)
    display(e.details())

The following code snippet displays the total retrieved bars count:

In [ ]:
ticks_count = 0
for b in bars:
    ticks_count += b.count
    
print('{0} tick bars retrieved from {1}'.format(len(bars), ticks_count))

### Step 4: Visualize data

#### 4.1 Retrieve intraday bars
In the following code snippets, the reply is reprocessed in a data frame in order to visualize the results with ease:

In [ ]:
# prepare the dataframe content
dates=[datetime.fromtimestamp(b.time_stamp.seconds) for b in bars]
opens = [b.open for b in bars]
highs = [b.high for b in bars]
lows = [b.low for b in bars]
closes = [b.close for b in bars]
volumes = [ts.volume for ts in bars]
counts = [ts.count for ts in bars]
vwaps = [ts.vwap for ts in bars]

In [ ]:
# create a pandas dataframe with: dates, bars and ticks count used for each bar
d = {'Date': dates, 'Open': opens, 'High': highs, 'Low' : lows,'Close': closes, 'Volume': volumes, 'Count': counts, 'Vwap': vwaps}
df = pd.DataFrame(data=d)
df = df.set_index('Date')
df

#### 4.2 Plot intraday bars with mplfinance

In [ ]:
# set number of bars
N = 100

In [ ]:
import mplfinance as mpf

mpf.plot(df.tail(N), figratio=(40,15),
         type = 'candle', 
         title=('{0}-min tick bars for {1}-{2} | Last {3} bars'.format(int(sampling/60),ticker,exchange,N)),
         volume = True, 
         tight_layout = True,
         style='yahoo'
        )

#### 4.3 Plot intraday bars with plotly

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name="Bars"), secondary_y=True)
fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name="Volume"), secondary_y=False)

fig.update_layout(title = '{0}-min bars for {1}-{2}'.format(int(sampling/60), ticker, exchange))
fig.update_xaxes(title_text = 'Date')
fig.layout.yaxis2.showgrid=False
fig.show()